In [ ]:
!pip install fastapi uvicorn cloudinary pyngrok flask werkzeug requests

# Step 2: Write Flask app code to a file
app_code = """
from flask import Flask, request, jsonify
import cloudinary
import cloudinary.uploader
import requests
import subprocess
import os
from werkzeug.utils import secure_filename

app = Flask(_name_)

# Cloudinary configuration
cloudinary.config(
    cloud_name='dczzbplxb',
    api_key='746916391552411',
    api_secret='gDxCEngH70PIEuNwoaBeaVeC8nY'
)

@app.route('/upload', methods=['POST'])
def upload_files():
    if 'image' not in request.files or 'cloth_url' not in request.form:
        return jsonify({"error": "Please provide an image file and a cloth URL"}), 400

    image_file = request.files['image']
    cloth_url = request.form['cloth_url']

    image_path = f"/tmp/{secure_filename(image_file.filename)}"
    cloth_path = f"/tmp/cloth_image.png"

    image_file.save(image_path)

    try:
        # Download the cloth image from the URL
        cloth_response = requests.get(cloth_url)
        if cloth_response.status_code != 200:
            return jsonify({"error": "Failed to download cloth image"}), 400
        with open(cloth_path, 'wb') as cloth_file:
            cloth_file.write(cloth_response.content)

        # Invoke the image processing script
        print("Running Invocation.py...")
        subprocess.check_output(f'python /content/ladi-vton/Invocation.py {image_path} {cloth_path}', shell=True)

        print("Running inference.py...")
        subprocess.check_output(
            f'cd /content/ladi-vton && python src/inference.py '
            f'--dataset vitonhd --vitonhd_dataroot "/content/ladi-vton/data" '
            f'--output_dir "output" --test_order unpaired --category all '
            f'--mixed_precision no --enable_xformers_memory_efficient_attention --use_png',
            shell=True
        )

        # Construct the correct output path based on the input image name
        image_base_name = os.path.splitext(image_file.filename)[0]
        output_image_path = f"/content/ladi-vton/output/unpaired/upper_body/{image_base_name}_0.png"

        if not os.path.exists(output_image_path):
            return jsonify({"error": "Output file not found"}), 500

        # Upload the result to Cloudinary
        upload_result = cloudinary.uploader.upload(output_image_path)
        return jsonify({"url": upload_result['url']})
    except subprocess.CalledProcessError as e:
        return jsonify({"error": f"Subprocess failed: {str(e)}"}), 500
    except FileNotFoundError as e:
        return jsonify({"error": f"File not found: {str(e)}"}), 500
    finally:
        # Clean up temporary files
        os.remove(image_path)
        os.remove(cloth_path)
        if os.path.exists(output_image_path):
            os.remove(output_image_path)

if _name_ == '_main_':
    app.run(host='0.0.0.0', port=5000)
"""

with open('app.py', 'w') as f:
    f.write(app_code)

# Step 3: Authenticate ngrok
from pyngrok import ngrok

authtoken = "2tKCurijzNcb4G1v8QYOvvcS4Wx_4irX46W2LzAUgRnb4nexv"
ngrok.set_auth_token(authtoken)

# Step 4: Open a tunnel to the Flask app on port 5000
public_url = ngrok.connect(5000)
print(f'Public URL: {public_url}')